In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip setuptools wheel
# !{sys.executable} -m pip install networkx numpy matplotlib pandas pydot

In [2]:
import networkx as nx
import itertools as it
import matplotlib.pyplot as plt
import math
import scipy
import pickle
import os

In [3]:
if not os.path.exists("Graphs"):
    os.mkdir("Graphs")
os.chdir("Graphs")
BaseDirectory = os.getcwd()

In [4]:
# print(BaseDirectory)

In [5]:
def _draw_graph(Graph):
    Dimension = 1
    FigDimension = 5
    fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
    nx.draw_circular(Graph, ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='white')
    axes.set_title("Graph 0")

In [6]:
def _draw_graph_list(GraphList, Names = None):
    if Names == None:
        Names = []
        for i in range(len(GraphList)):
            Names.append(f"Graph {i}")
    if len(GraphList) <= 0:
        return
    elif len(GraphList) == 1:
        Dimension = 1
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        nx.draw_circular(GraphList[0], ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='silver')
        axes.set_title("Graph 0")
    else:
        Dimension = math.ceil(math.sqrt(len(GraphList)))
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        GraphCounter=0
        for x in range(0,Dimension):
            for y in range(0,Dimension):
                if GraphCounter < len(GraphList):
                    axes[x,y].set_title(Names[GraphCounter])
                    nx.draw_circular(GraphList[GraphCounter], ax=axes[x,y], with_labels=True, edge_color='black', node_color='black', font_color='silver')
                    GraphCounter+=1
                else:
                    nx.draw(nx.null_graph(), ax=axes[x,y])
    plt.tight_layout()
    plt.show()
    plt.close()
    return

In [7]:
def _save_graph(Graph, FileName = "Default", Name = None):
    if Name == None:
        Name = "Graph"
    fig,axes = plt.subplots(nrows=1, ncols=1, figsize=(math.ceil(math.sqrt(Graph.number_of_nodes())),math.ceil(math.sqrt(Graph.number_of_nodes()))))
    nx.draw_circular(Graph, ax=axes, with_labels=True, edge_color="black", node_color="Black", font_color="silver")
    plt.tight_layout()
    plt.savefig(f"{FileName}.svg")
    plt.close()

In [8]:
def _save_graph_list(GraphList, FileName = "Default", Names = None):
    if Names == None:
        Names = []
        for i in range(len(GraphList)):
            Names.append(f"Graph {i}")
    if len(GraphList) <= 0:
        return
    elif len(GraphList) == 1:
        Dimension = 1
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        nx.draw_circular(GraphList[0], ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='silver')
        axes.set_title("Graph 0")
    else:
        Dimension = math.ceil(math.sqrt(len(GraphList)))
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        GraphCounter=0
        for x in range(0,Dimension):
            for y in range(0,Dimension):
                if GraphCounter < len(GraphList):
                    axes[x,y].set_title(Names[GraphCounter])
                    nx.draw_circular(GraphList[GraphCounter], ax=axes[x,y], label=f"Graph {GraphCounter}", with_labels=True, edge_color='black', node_color='black', font_color='silver')
                    GraphCounter+=1
                else:
                    nx.draw(nx.null_graph(), ax=axes[x,y])
    plt.tight_layout()
    plt.savefig(f"{FileName}.svg")
    plt.close()
    return

In [9]:
def _get_largest_subgraph_of_both(Graph1, Graph2):
#     
    
    if Graph1.size() > Graph2.size():
        LargestGraph = Graph1
        SmallestGraph = Graph2
    else:
        LargestGraph = Graph2
        SmallestGraph = Graph1
    
    if SmallestGraph.number_of_nodes() < 1:
        return nx.null_graph()
    
    LargestSubgraph = nx.path_graph(1)
    for SubgraphSize in range(SmallestGraph.size(), 0, -1):
        for Edges in it.combinations(SmallestGraph.edges(), SubgraphSize):
            HostLineGraph = nx.line_graph(LargestGraph)
            
            if nx.algorithms.isomorphism.GraphMatcher(LargestGraph, nx.edge_subgraph(SmallestGraph, Edges).copy()).subgraph_is_monomorphic():
                LargestSubgraph = nx.edge_subgraph(SmallestGraph, Edges).copy()
                break
        if LargestSubgraph.size() > 0:
            break
            
    return LargestSubgraph

In [10]:
def _save_coloring(ColoringList, Intersection = None, HostGraph = None, FileName = "Default"):
#     This function saves specifically a red blue coloring
    
    fig,axes = plt.subplots(nrows=1, ncols=4, figsize=(10,2.5))
    nx.draw_circular(ColoringList[0], ax=axes[0], with_labels=True, edge_color="red", node_color='black', font_color='white')
    axes[0].set_title("Red Subgraph")
    
    nx.draw_circular(ColoringList[1], ax=axes[1], with_labels=True, edge_color="blue", node_color='black', font_color='white')
    axes[1].set_title("Blue Subgraph")
    
    if Intersection == None:
        Intersection = _get_largest_subgraph_of_both(ColoringList[0], ColoringList[1])
    nx.draw_circular(Intersection, ax=axes[2], with_labels=True, edge_color="violet", node_color='black', font_color='white')
    axes[2].set_title("Intersection")
    
    if HostGraph == None:
        HostGraph = nx.compose(ColoringList[0], ColoringList[1])
    nx.draw_circular(HostGraph, ax=axes[3], with_labels=True, edge_color="black", node_color='black', font_color='white')
    axes[3].set_title("Host")
    
    plt.tight_layout()
    plt.savefig(f"{FileName}.svg")
    plt.close()
    return

In [13]:
def _all_edge_induced_subgraphs_generator(Host):
    
#     Iterate through selecting 0, 1, 2, ... edges that will be selected from the edge set of the host graph
    for NumEdgesChosen in range(Host.size()+1):
        
#         For each given number of edges that will be selected, select each combination of that many edges from the host graph
        for Edges in it.combinations(Host.edges(),NumEdgesChosen):
        
#             Add the respective edge-induced subgraph to the list of all subgraphs
            yield nx.edge_subgraph(Host, Edges).copy()

In [14]:
def _return_unique_graphs(GraphList):
#     This function takes in a list of graphs, and returns a list of unique graphs from the input list, taking graph isomorphism as the equivalence relation
    
#     Initialize the list of unique graphs to an empty list
    UniqueGraphs = []
    
#     Iterate over the entire list of graphs
    for Graph in GraphList:
        
#         First assume that the graph is unique up to subgraph isomorphism
        Unique = True
    
#         Iterate through each graph already in the list of unique subgraphs, and if there is a graph in the list of unique subgraphs that is isomorphic to the potentially unique subgraph, we note that and break out of the loop
        for UniqueGraph in UniqueGraphs:
            if nx.is_isomorphic(Graph, UniqueGraph):
                Unique = False
                break
                
#         If none of the already determined unique subgraphs are isomorphic to the held subgraph, then we can safely determine that it is unique up to isomorphism
        if Unique:
            UniqueGraphs.append(Graph)
            
    return UniqueGraphs

In [15]:
def _get_unique_edge_induced_subgraphs(Host):
#     This funciton is essentially a wrapper for returning only the unique subgraphs of an input graph
    return _return_unique_graphs(_all_edge_induced_subgraphs_generator(Host))

In [16]:
def _get_all_colorings(Host, SubgraphList = None):
#     This function takes in a host graph, and a list of subgraphs (or calculates all of them), and returns a list of all red blue colorings associated with coloring every subgraph in the list red and the complement (in the host) blue
#     This function returns a list of tuples where the first index is the red subgraph, and the second index is the blue subgraph
    
#     Initialize the list of colorings to an empty list
    Colorings = []
    
#     If the list of subgraphs isn't passed into this funciton, this funciton will calculate the list of all non-isomorphic subgraphs
    if SubgraphList == None:
        SubgraphList = _get_unique_edge_induced_subgraphs(Host)
    
#     Make the induced colorings and put them into the list of colorings
    for RedColoring in SubgraphList:
        BlueColoring = nx.edge_subgraph(Host, set(Host.edges())-set(RedColoring.edges())).copy()
        Colorings.append(tuple((RedColoring, BlueColoring)))
        
    return Colorings

In [17]:
def _return_total_ordering(GraphList):
    AdjacencyList = ""
    GraphID = 0
    for Graph in GraphList:
        AdjacencyList += f"{GraphID}"
        HostGraphID = 0
        for HostGraph in GraphList:
            if nx.algorithms.isomorphism.GraphMatcher(HostGraph, Graph).subgraph_is_monomorphic():
                AdjacencyList += f" {HostGraphID}"
            HostGraphID += 1
        AdjacencyList += "\n"
        GraphID += 1
    TotalOrdering = nx.parse_adjlist(AdjacencyList.splitlines(), create_using=nx.DiGraph(), nodetype=int)
    TotalOrdering.remove_edges_from(nx.selfloop_edges(TotalOrdering))
    return TotalOrdering

In [18]:
def _get_poset(GraphList, Ordering = None):
    if Ordering == None:
        Ordering = _return_total_ordering(GraphList)
    AdjacencyList = ""
    GraphID = 0
    for Graph in GraphList:
        AdjacencyList += f"{GraphID}"
        HostGraphID = 0
        for HostGraph in GraphList:
            if HostGraph.size() == (Graph.size()+1):
                if Ordering.has_edge(GraphID, HostGraphID):
                    AdjacencyList += f" {HostGraphID}"
            HostGraphID += 1
        AdjacencyList += "\n"
        GraphID += 1
    return nx.parse_adjlist(AdjacencyList.splitlines(), create_using=nx.DiGraph())

In [19]:
def _get_graph_from_file_name(GraphName):
    if "K_" in GraphName:
        if "," in GraphName:
            n,m=FileName.split(".",1)[0].split("K_",1)[1].split(",")
            n=int(n)
            m=int(m)
            return nx.complete_multipartite_graph(n,m)
        else:
            n=FileName.split(".",1)[0].split("K_",1)[1]
            n=int(n)
            return nx.complete_graph(n)
    elif "C_" in GraphName:
            n=FileName.split(".",1)[0].split("C_",1)[1]
            n=int(n)
            return nx.cycle_graph(n)
    elif "P_" in GraphName:
            n=FileName.split(".",1)[0].split("P_",1)[1]
            n=int(n)
            return nx.path_graph(n)

In [20]:
# Build the unique subgraphs of complete bipartites

MaxN = 4
MaxM = 4

for n in range(1,MaxN+1):
    for m in range(n,MaxM+1):
        GraphName = f"K_{n},{m}"
        
        if not os.path.exists(f"{GraphName}"):
            os.mkdir(f"{GraphName}")
        os.chdir(f"{GraphName}")
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.complete_multipartite_graph(n,m)
        
#         InducedGraphs = _get_all_edge_induced_subgraphs(Host)
#         print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of the chosen graph.")
        
        UniqueGraphs = _get_unique_edge_induced_subgraphs(Host)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)
        os.chdir(BaseDirectory)

Inspecting: K_1,1
     There are 2 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_1,2
     There are 3 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_1,3
     There are 4 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_1,4
     There are 5 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_2,2
     There are 6 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_2,3
     There are 12 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_2,4
     There are 21 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_3,3
     There are 26 distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.
Inspecting: K_3,4
     There are 76 distinct subgraphs up to isomorphism with at leas

In [21]:
# Build the unique subgraphs of complete graphs

MaxK = 5

for k in range(1,MaxK+1):
        GraphName = f"K_{k}"
        
        if not os.path.exists(f"{GraphName}"):
            os.mkdir(f"{GraphName}")
        os.chdir(f"{GraphName}")
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.complete_graph(k)
        
#         InducedGraphs = _get_all_edge_induced_subgraphs(Host)
#         print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _get_unique_edge_induced_subgraphs(Host)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)
        os.chdir(BaseDirectory)

Inspecting: K_1
     There are 1 distinct subgraphs up to isomorphism with at least 1 edge of K_1.
Inspecting: K_2
     There are 2 distinct subgraphs up to isomorphism with at least 1 edge of K_2.
Inspecting: K_3
     There are 4 distinct subgraphs up to isomorphism with at least 1 edge of K_3.
Inspecting: K_4
     There are 11 distinct subgraphs up to isomorphism with at least 1 edge of K_4.
Inspecting: K_5
     There are 34 distinct subgraphs up to isomorphism with at least 1 edge of K_5.


In [22]:
# Build the unique subgraphs of cycle graphs

MaxN = 9

for n in range(1,MaxN+1):
        GraphName = f"C_{n}"
        
        if not os.path.exists(f"{GraphName}"):
            os.mkdir(f"{GraphName}")
        os.chdir(f"{GraphName}")
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.cycle_graph(n)
        
#         InducedGraphs = _get_all_edge_induced_subgraphs(Host)
#         print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _get_unique_edge_induced_subgraphs(Host)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)
        os.chdir(BaseDirectory)

Inspecting: C_1
     There are 2 distinct subgraphs up to isomorphism with at least 1 edge of C_1.
Inspecting: C_2
     There are 2 distinct subgraphs up to isomorphism with at least 1 edge of C_2.
Inspecting: C_3
     There are 4 distinct subgraphs up to isomorphism with at least 1 edge of C_3.
Inspecting: C_4
     There are 6 distinct subgraphs up to isomorphism with at least 1 edge of C_4.
Inspecting: C_5
     There are 8 distinct subgraphs up to isomorphism with at least 1 edge of C_5.
Inspecting: C_6
     There are 12 distinct subgraphs up to isomorphism with at least 1 edge of C_6.
Inspecting: C_7
     There are 16 distinct subgraphs up to isomorphism with at least 1 edge of C_7.
Inspecting: C_8
     There are 23 distinct subgraphs up to isomorphism with at least 1 edge of C_8.
Inspecting: C_9
     There are 31 distinct subgraphs up to isomorphism with at least 1 edge of C_9.


In [23]:
# Build the unique subgraphs of cycle graphs

MaxN = 9

for n in range(1,MaxN+1):
        GraphName = f"P_{n}"
        
        if not os.path.exists(f"{GraphName}"):
            os.mkdir(f"{GraphName}")
        os.chdir(f"{GraphName}")
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.path_graph(n)
        
#         InducedGraphs = _get_all_edge_induced_subgraphs(Host)
#         print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _get_unique_edge_induced_subgraphs(Host)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)
        os.chdir(BaseDirectory)

Inspecting: P_1
     There are 1 distinct subgraphs up to isomorphism with at least 1 edge of P_1.
Inspecting: P_2
     There are 2 distinct subgraphs up to isomorphism with at least 1 edge of P_2.
Inspecting: P_3
     There are 3 distinct subgraphs up to isomorphism with at least 1 edge of P_3.
Inspecting: P_4
     There are 5 distinct subgraphs up to isomorphism with at least 1 edge of P_4.
Inspecting: P_5
     There are 7 distinct subgraphs up to isomorphism with at least 1 edge of P_5.
Inspecting: P_6
     There are 11 distinct subgraphs up to isomorphism with at least 1 edge of P_6.
Inspecting: P_7
     There are 15 distinct subgraphs up to isomorphism with at least 1 edge of P_7.
Inspecting: P_8
     There are 22 distinct subgraphs up to isomorphism with at least 1 edge of P_8.
Inspecting: P_9
     There are 30 distinct subgraphs up to isomorphism with at least 1 edge of P_9.


In [24]:
# Build the colorings, and  intersections
for DirName in os.listdir(os.getcwd()):
    if os.path.isdir(DirName):
        if "ipynb" not in DirName:
            os.chdir(DirName)
            for FileName in os.listdir(os.getcwd()):
                if ".UniqueGraphs.pickle" in FileName:
                    HostGraph = _get_graph_from_file_name(FileName)
                    GraphName = FileName.split(".",1)[0]
                    print(f"Inspecting {GraphName}")

            #         Pick up the list of unique subgraphs
                    if not os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
                        continue
                    UniqueSubgraphs = []
                    CommonIntersectionSubgraphs = []
                    with open(f"{GraphName}.UniqueGraphs.pickle", "rb") as InputFile:
                        UniqueSubgraphs = pickle.load(InputFile)

            #         Build a directory (if need be) and move into it for the colorings to be saved
                    if not os.path.exists("Colorings"):
                        os.mkdir("Colorings")
                    os.chdir("Colorings")

            #         Determine all colorings, and save images of the colorings (and their intersection)
                    Colorings = _get_all_colorings(HostGraph)
                    ColoringIntersections = []
                    ColoringID = 0
                    for Coloring in Colorings:
                        ColoringIntersections.append(_get_largest_subgraph_of_both(Coloring[0], Coloring[1]))
                        _save_coloring(Coloring, ColoringIntersections[ColoringID], HostGraph, f"{GraphName}.Coloring.{ColoringID}")
                        ColoringID += 1

#                     Save the coloring graph list just in case we need it later
                    with open(f"{GraphName}.Colorings.pickle", "wb") as OutFile:
                        pickle.dump(Colorings, OutFile)

#                     Now return to the start
                    os.chdir(BaseDirectory)
                    os.chdir(GraphName)
        
#                     Build a directory (if need be) and move into it for the coloring poset to be saved
                    if not os.path.exists("Coloring Intersections"):
                        os.mkdir("Coloring Intersections")
                    os.chdir("Coloring Intersections")
                    
                    ColoringIntersections = _return_unique_graphs(ColoringIntersections)
                    _save_graph_list(ColoringIntersections, f"{GraphName}.Coloring.Intersections")
                    
                    ColoringIntersectionPoset = _get_poset(ColoringIntersections)
                    _save_graph(ColoringIntersectionPoset, f"{GraphName}.Coloring.Poset")
                    
#                     Save the unique coloring intersections in case we need it later
                    with open(f"{GraphName}.Unique.Coloring.Intersections.pickle", "wb") as OutFile:
                        pickle.dump(_return_unique_graphs(ColoringIntersections), OutFile)
            
#                     Save the poset of coloring intersections in case we need it later
                    with open(f"{GraphName}.Coloring.Intersection.Poset.pickle", "wb") as OutFile:
                        pickle.dump(ColoringIntersectionPoset, OutFile)

                    os.chdir(BaseDirectory)

Inspecting K_1,1
Inspecting P_8
Inspecting P_1
Inspecting P_6
Inspecting P_7
Inspecting K_3,3
Inspecting K_3,4
Inspecting P_9
Inspecting K_4,4
Inspecting C_2
Inspecting K_1
Inspecting C_5
Inspecting C_4
Inspecting C_3
Inspecting K_1,2
Inspecting P_5
Inspecting P_2
Inspecting P_3
Inspecting P_4
Inspecting K_1,4
Inspecting K_1,3
Inspecting K_2,2
Inspecting K_2,4
Inspecting K_2,3
Inspecting C_6
Inspecting K_5
Inspecting C_1
Inspecting K_2
Inspecting C_8
Inspecting C_9
Inspecting K_3
Inspecting C_7
Inspecting K_4


In [25]:
print("Done")

Done
